In [24]:
!pip install google-generativeai langchain langchain-community langchain-openai langchain-google-genai faiss-cpu pypdf sentence-transformers


In [39]:
import google.generativeai as genai
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [26]:
GEMINI_API_KEY = "AIzaSyCn54pbAZGimcVeiFdJmSYuqu0LhSjKxxxxx"
genai.configure(api_key=GEMINI_API_KEY)

In [42]:
print("Các model Gemini có sẵn:")
for model in genai.list_models():
    if "gemini" in model.name.lower():
        print(f"- {model.name}")

Các model Gemini có sẵn:
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-001
- models/gemini-1.5-pro-002
- models/gemini-1.5-pro
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash-001
- models/gemini-1.5-flash-001-tuning
- models/gemini-1.5-flash
- models/gemini-1.5-flash-002
- models/gemini-1.5-flash-8b
- models/gemini-1.5-flash-8b-001
- models/gemini-1.5-flash-8b-latest
- models/gemini-1.5-flash-8b-exp-0827
- models/gemini-1.5-flash-8b-exp-0924
- models/gemini-2.5-pro-exp-03-25
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-

In [43]:
print("Đang tải tài liệu PDF...")
pdf_path = "/content/introduction_to_data_science_2017.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Đã tải {len(docs)} đoạn văn bản từ PDF.")

Đang tải tài liệu PDF...
Đã tải 227 đoạn văn bản từ PDF.


In [46]:
print("Đang tạo vector embeddings...")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding)

Đang tạo vector embeddings...


In [47]:
def retrieve_docs(query, k=3):
    docs = vectorstore.similarity_search(query, k=k)
    return docs

def generate_response(query, context):
    prompt = f"""
Sử dụng các thông tin dưới đây để trả lời câu hỏi.
Nếu bạn không biết câu trả lời, hãy nói rằng bạn không biết, đừng cố gắng bịa ra câu trả lời.

Context:
{context}

Câu hỏi: {query}
Câu trả lời:
"""
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Lỗi khi tạo câu trả lời: {str(e)}"

print("Hệ thống RAG đã sẵn sàng. Nhập 'exit' hoặc 'quit' để kết thúc.")


Hệ thống RAG đã sẵn sàng. Nhập 'exit' hoặc 'quit' để kết thúc.


In [ ]:
while True:
    query = input("Bạn: ")
    if query.lower() in ["exit", "quit"]:
        break

    try:
        retrieved_docs = retrieve_docs(query)
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])

        response = generate_response(query, context)

        print("Chatbot:", response)

    except Exception as e:
        print(f"Lỗi: {e}")
        print("Hãy thử câu hỏi khác.")

Bạn: hello
Chatbot: Tôi không biết.
Bạn: sách này là gì
Chatbot: Dựa trên thông tin được cung cấp, đây là một cuốn sách về nhận dạng đối tượng trực quan, với sự quan tâm đặc biệt đến phục hồi tư thế người và phân tích hành vi từ dữ liệu đa phương thức.
Bạn: tóm tắt cuốn sách
Chatbot: Cuốn sách này thảo luận về học máy có giám sát, đặc biệt là các kỹ thuật phân tích tình cảm. Nó bao gồm các chủ đề sau:

*   **Học máy có giám sát:** Sử dụng dữ liệu được gán nhãn để huấn luyện mô hình dự đoán.
*   **Phân tích tình cảm:** Xác định tình cảm biểu hiện trong một đoạn văn bản (ví dụ: tích cực, tiêu cực, trung tính).
*   **Các thuật toán phân loại:** Bao gồm Naive Bayes, Support Vector Machines (SVM) và K-Nearest Neighbors (KNN).
*   **Đánh giá mô hình:** Sử dụng các số liệu như độ chính xác, ma trận nhầm lẫn (confusion matrix) để đánh giá hiệu suất của các mô hình phân loại.
*   **Cross-validation:** Kỹ thuật chia dữ liệu thành nhiều phần (folds) để huấn luyện và đánh giá mô hình một cách khác